### This is the jupyter notebook for sentiment analysis of the tweets using the Twitter API
### FYP project
### Author: Muhammad Abu Bakar
### Author: Muhammad Faiz Hassan


In [85]:
# 1. Import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tweepy
from tweepy import OAuthHandler


In [86]:
consumer_key = "gxoci9hQ7adNawtYj4r5J459y"
consumer_secret = "Vn1HMAVwH7Bl3MZXlwGA5kaCOFvs9ZOJ4b3Wg1JRSapU2kbA4G"
access_token = "930155901666971648-lrpZsfrmdQQihbTLUZKs6pASEdNS5ek"
access_token_secret = "sjiascsBPd8UBkqPNPUYbaL0RoLxnD9gikEmmRGEusftr"

# configured api's with consumer k

# Use the above credentials to authenticate the API.

auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)
print(api)

In [94]:
# 3.To get the tweets in a Proper format, first lets create a Dataframe to store the extracted data.

df = pd.DataFrame(columns=["date","User","IsVerified","Tweet","Likes","RT",'User_location'])

# df['date'] = df['date'].apply(lambda a: pd.to_datetime(a).date())
# df['date'] = pd.to_datetime(df['date'])
# df.dtypes
# df['date'] = df['date'].dt.tz_localize(None)
print(df)



TypeError: __init__() got an unexpected keyword argument 'axis'

## We will use api as api.search inside this tweepy cursor.


In [101]:
# 4.Write a Function to extract tweets:

# We will Use **tweepy.cursor()** because we want to extract a larger number of tweets i.e over 100,500 etc


def get_tweets(Topic,Count):    
    i=0
    for tweet in tweepy.Cursor(api.search_tweets, q=Topic,count=10, lang="en",exclude='retweets').items():
        print(i, end='\r')
        date=tweet.created_at
        df.loc[i,"date"] = date.strftime("%Y-%m-%d")
        # print('hello')
        df.loc[i,"User"] = tweet.user.name
        df.loc[i,"IsVerified"] = tweet.user.verified
        df.loc[i,"Tweet"] = tweet.text
        df.loc[i,"Likes"] = tweet.favorite_count
        df.loc[i,"RT"] = tweet.retweet_count
        df.loc[i,"User_location"] = tweet.user.location
        #df.to_csv("TweetDataset.csv",index=False)
        df.to_excel('{}.xlsx'.format("TweetDataset"),index=False)   ## Save as Excel
        i=i+1
        if i>Count:
            break
        else:
            pass

# take input from user
Topic = input("Enter Keyword/Tag to search about: ")
# Topic=["Pakistan"]
get_tweets(Topic , Count=10)

Unexpected parameter: exclude


Unexpected parameter: exclude


In [102]:
# Showing the data the Data
df.head(8)

,date,User,IsVerified,Tweet,Likes,RT,User_location
0,2022-11-25,Saba Khan,False,110% agreed... main root cause of all the brut...,0,0,Islamabad
1,2022-11-25,★ Ms Potting Shed 🇺🇦,False,@Jennife10651535 @greendoondoon I'm always goo...,0,0,East Sussex
2,2022-11-25,Abdullah Khakel,False,@ImranARaja1 Politics aside he is a total emba...,0,0,
3,2022-11-25,Steve 🇾🇪,False,@HamzaD86 Pakistan are better than England (fo...,0,0,Manchester.
4,2022-11-25,Syed Mazhar,False,Foreign Minister of Pakistan @BBhuttoZardari s...,0,0,"Karachi, Pakistan"
5,2022-11-25,ShafaqFatima,False,@PTIofficial Corruption free Pakistan with the...,0,0,In My Thoughts
6,2022-11-25,Abdullah Qaisar,False,"@TheBarmyArmy W’alaikum As Salaam, and welcome...",0,0,
7,2022-11-25,David Gillespie,False,@AnasSarwar @heraldscotland You could not make...,0,0,


In [103]:
# Function to Clean the Tweet.

import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|([RT])', ' ', str(tweet).lower()).split())

# We only want the Text so :

# (@[A-Za-z0-9]+)   : Delete Anything like @hello @Letsupgrade etc
# ([^0-9A-Za-z \t]) : Delete everything other than text,number,space,tabspace
# (\w+:\/\/\S+)     : Delete https://
# ([RT]) : Remove "RT" from the tweet

In [114]:
from textblob import TextBlob
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [ ]:
#Function to Pre-process data for Worlcloud:here we are removing the words present in Topic from the Corpus so they dont come in WordCloud.
# Ex : Topic is "Arsenal vs United", we want to remove "Arsenal" "vs" "United" from the WordCloud.

def prepCloud(Topic_text,Topic):
    Topic = str(Topic).lower()
    Topic=' '.join(re.sub('([^0-9A-Za-z \t])', ' ', Topic).split())
    Topic = re.split("\s+",str(Topic))
    stopwords = set(STOPWORDS)
    stopwords.update(Topic) ### Add our topic in Stopwords, so it doesnt appear in wordClous
    ###
    text_new = " ".join([txt for txt in Topic_text.split() if txt not in stopwords])
    return text_new
